In [ ]:
! pip install openai

In [ ]:
! pip install stanza

### ASPEKTU IZGŪŠANA

In [ ]:
# OpenAI. GPT-4o. Uzvedne: kā sasaistīt zemāk dotos kodus un izlabot manu kodu tā, lai pēc katras uzvednes tiktu izgūts rezultāts kurš uzreiz tiek salīdzināts ar attiecīgo pareizo atbildi, nesalīdzinot ar ID, un tad pareizi tiek aprēķinātas metrikas? https://chatgpt.com/ [izmantots 2025-04-24]
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import xml.etree.ElementTree as ET
import pandas as pd
import json
import re
import openai
from tqdm import tqdm

# OpenRouter. Sample code and API for DeepSeek V3 0324 (free). Tiešsaiste. OpenRouter. Pieejams: https://openrouter.ai/deepseek/deepseek-chat-v3-0324:free/api. [skatīts 2025-03-29].
# OpenAI. ChatGPT o3-mini-high. Uzvedne: kā man uzlabot šo kodu, lai tas izmantotu no XML izgūtos teikumus, aspektus un noskaņojumus noskaņojuma noteikšanai aspektiem un vēl novērtētu modeļa veiktspēju? https://chatgpt.com/ [izmantots 2025-03-29]

def parse_xml_aspect_extraction(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    data = []
    for sentence in root.iter('sentence'):
        if "OutOfScope" in sentence.attrib and sentence.attrib["OutOfScope"].upper() == "TRUE":
          continue
        text = sentence.find('text').text.strip()
        opinions = sentence.find('Opinions')
        aspects = []
        if opinions is not None:
            for opinion in opinions.findall('Opinion'):
                target = opinion.get('target')
                if target and target.lower() != "null":
                  aspects.append(target.lower())
        data.append({
            'id': sentence.get('id'),
            'text': text,
            'aspects': aspects
        })
    return pd.DataFrame(data)

client = openai.OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=""
)
def extract_aspects(sentence_id, sentence):
    prompt = (
        "Your task is to do aspect term extraction.\n"
        "Given a sentence, extract all explicit aspect terms from a sentence.\n"
        "Return the result as a single JSON object with key 'results'.\n"
        "If there are no aspect terms in the sentence, return an empty list for 'results'.\n"
        "Extract the aspect terms exactly as they appear in the sentence, preserving their original form and grammatical case.\n"
        "Do not provide any additional text or explanation, do not guess implicit aspects, do not extract adjectives, verbs or general sentiment expressions.\n"
        "Do not include broad or vague terms (e.g., 'cena', 'diena') and always preserve the full noun phrase (e.g., 'ķiploku grauzdiņi', not just 'grauzdiņi')\n"
        "Respond ONLY with valid JSON.\n"
        "Example format:\n"
        "{\"results\": [\"ēdiens\", \"apkalpošana\"]}\n"
        "{\"results\": []}\n\n"
        f"Sentence: \"{sentence}\""
    )
    try:
        completion = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model="deepseek/deepseek-chat"
        )
        content = completion.choices[0].message.content.strip()
        match = re.search(r"\{.*\}", content, re.DOTALL)
        if not match:
            raise ValueError("No JSON in response.")
        parsed = json.loads(match.group(0))
        return parsed.get("results", [])

# OpenAI. GPT-4o. Uzvedne: varbūt būtu labāk šīs nepareizās atbildes ar exception kļūdām nesaglabāt vispār JSON failā bet kaut kur atsevišķi, jo tad vienkārši JSON failā nebūs kļūdainu rezultātu, kurus nevarēs nolasīt un ar šo rindiņu if sentence_id not in prediction_dict: uzreiz tiks uzskatīta kā kļūda https://chatgpt.com/ [izmantots 2025-04-24]
    except Exception as e:
        print(f"[ERROR] ID {sentence_id} — {e}")
        with open("llm_errors_aspects.log", "a", encoding="utf-8") as f:
            f.write(f"{sentence_id} — {e}\n")
            f.write(f"Original response: {completion}\n\n")
        return None

# OpenAI. GPT-4o. Uzvedne: kā izlabot manā kodā metriku aprēķināšanas funkciju, lai tiktu iegūti TP, FP un FN? https://chatgpt.com/ [izmantots 2025-04-26]
def evaluate_live(test_data):
    TP = 0
    FP = 0
    FN = 0
    skipped = 0
    mismatches = []

    for _, row in tqdm(test_data.iterrows(), total=len(test_data), desc="Processing"):

        current_TP = 0
        current_FP = 0
        current_FN = 0

        sentence_id = row["id"]
        sentence = row["text"]
        true_aspects = [a.lower() for a in row["aspects"]]
        pred_aspects = extract_aspects(row["id"], row["text"])

        if pred_aspects is None:
            skipped += 1
            FN += len(true_aspects)
            continue

        pred_aspects = [a.lower() for a in pred_aspects]

        true_set = set(true_aspects)
        pred_set = set(pred_aspects)

        current_TP += len(true_set & pred_set)   # Pareizi atrastie aspekti
        current_FP += len(pred_set - true_set)   # Nepareizi atrastie aspekti (kļūdaini izdomāti)
        current_FN += len(true_set - pred_set)   # Pazudušie aspekti (neatrastie)

        TP += current_TP
        FP += current_FP
        FN += current_FN

        if current_FP > 0 or current_FN > 0:
            mismatches.append({
                'id': sentence_id,
                'text': sentence,
                'true_aspects': true_aspects,
                'predicted_aspects': pred_aspects,
                'FP': list(pred_set - true_set),
                'FN': list(true_set - pred_set)
            })

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    print(f"\nSalīdzināti {len(test_data)} teikumi (izlaisti {skipped})")
    print(f"True Positives (TP): {TP}")
    print(f"False Positives (FP): {FP}")
    print(f"False Negatives (FN): {FN}")
    print(f"\nPrecision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1 score:  {f1:.4f}")

    with open("errors_fp_fn.json", "w", encoding="utf-8") as f:
        json.dump(mismatches, f, ensure_ascii=False, indent=2)

    print("\nFP un FN kļūdu saraksts saglabāts uz 'errors_fp_fn.json'")

test_data = parse_xml_aspect_extraction("testGOLD2016_1.xml")
evaluate_live(test_data)

### ASPEKTU NOSKAŅOJUMU NOTEIKŠANA

In [ ]:
# OpenRouter. Sample code and API for DeepSeek V3 0324 (free). Tiešsaiste. OpenRouter. Pieejams: https://openrouter.ai/deepseek/deepseek-chat-v3-0324:free/api. [skatīts 2025-03-29].
# OpenAI. ChatGPT o3-mini-high. Uzvedne: kā man uzlabot šo kodu, lai tas izmantotu no XML izgūtos teikumus, aspektus un noskaņojumus noskaņojuma noteikšanai aspektiem un vēl novērtētu modeļa veiktspēju? https://chatgpt.com/ [izmantots 2025-03-29]
# https://console.groq.com/docs/quickstart
# OpenAI. GPT-4o. Uzvedne: kā sasaistīt zemāk dotos kodus un izlabot manu kodu tā, lai pēc katras uzvednes tiktu izgūts rezultāts kurš uzreiz tiek salīdzināts ar attiecīgo pareizo atbildi, nesalīdzinot ar ID, un tad pareizi tiek aprēķinātas metrikas? https://chatgpt.com/ [izmantots 2025-04-24]

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import xml.etree.ElementTree as ET
import pandas as pd
import json
import re
import openai
from tqdm import tqdm
from itertools import zip_longest
from tqdm import tqdm
import json

def parse_xml_sentiment_pred(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    data = []
    for sentence in root.iter('sentence'):
        if "OutOfScope" in sentence.attrib and sentence.attrib["OutOfScope"].upper() == "TRUE":
          continue
        text = sentence.find('text').text.strip()
        opinions = sentence.find('Opinions')
        aspects = []
        sentiments = []
        if opinions is not None:
            for opinion in opinions.findall('Opinion'):
                target = opinion.get('target')
                polarity = opinion.get('polarity')
                if target and target.lower() != "null":
                  aspects.append(target.lower())
                  sentiments.append(polarity)
        if aspects:
          data.append({
              'id': sentence.get('id'),
              'text': text,
              'aspects': aspects,
              'sentiments': sentiments
          })
    return pd.DataFrame(data)

client = openai.OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=""
)
def predict_sentiment_only_labels(sentence_id, sentence, aspects):
    aspects_str = ", ".join(f'\"{asp}\"' for asp in aspects)
    prompt = (
        "Your task is to perform aspect-based sentiment analysis.\n"
        "Classify the sentiment for each of the listed aspect terms in the given sentence.\n"
        "Use only one of the following labels: positive, negative, neutral.\n"
        "Return a singe JSON object with key 'results' and value as a list of sentiment labels in the same order as the aspects provided.\n"
        "Respond ONLY with valid JSON like: {\"results\": [\"positive\", \"neutral\"]}\n"
        "Do NOT include any extra text or explanations.\n"
        f"Sentence: \"{sentence}\"\n"
        f"Aspect terms: [{aspects_str}]"
    )
    try:
        completion = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model="deepseek/deepseek-chat"
        )
        content = completion.choices[0].message.content.strip()
        match = re.search(r"\{.*\}", content, re.DOTALL)
        if not match:
            raise ValueError("No JSON in response.")
        parsed = json.loads(match.group(0))
        results = parsed.get("results", [])
        return results

# OpenAI. GPT-4o. Uzvedne: varbūt būtu labāk šīs nepareizās atbildes ar exception kļūdām nesaglabāt vispār JSON failā bet kaut kur atsevišķi, jo tad vienkārši JSON failā nebūs kļūdainu rezultātu, kurus nevarēs nolasīt un ar šo rindiņu if sentence_id not in prediction_dict: uzreiz tiks uzskatīta kā kļūda https://chatgpt.com/ [izmantots 2025-04-24]
    except Exception as e:
        print(f"[ERROR] ID {sentence_id} — {e}")
        with open("llm_errors_sentiments.log", "a", encoding="utf-8") as f:
            f.write(f"{sentence_id} — {e}\n")
            if 'completion' in locals():
                f.write(f"Original response: {completion}\n\n")
            else:
                f.write("Original response: N/A (no response returned)\n\n")
        return None
# OpenAI. GPT-4o. Uzvedne: kā izlabot manā kodā metriku aprēķināšanas funkciju, lai tiktu iegūti TP, FP un FN? https://chatgpt.com/ [izmantots 2025-04-26]
# OpenAI. ChatGPT o4-mini-high. Uzvedne: kā how to fix evaluate_sentiment_classification, so that it checks if pred_sentiments is None, checks if sentiment is valid (positive, negative, neutral), checks if there are missing sentiments and if there are excess sentiments, checks if sentiment is correct and matches true sentiment https://chatgpt.com/ [izmantots 2025-05-07]

def evaluate_sentiment_classification(test_data):
    TP = FP = FN = 0
    errors = []
    skipped = 0
    valid_labels = {"positive", "negative", "neutral"}

    for _, row in tqdm(test_data.iterrows(), total=len(test_data), desc="Processing"):
        true_sentiments = [s.lower() for s in row["sentiments"]]
        pred_sentiments = predict_sentiment_only_labels(row["id"], row["text"], row["aspects"])

        if pred_sentiments is None:
            FN += len(true_sentiments)
            skipped += 1
            continue

        for true_s, pred_s in zip_longest(true_sentiments, pred_sentiments, fillvalue=None):
            if pred_s is None:
                FN += 1
                errors.append({
                    "id": row["id"],
                    "aspect": row["aspects"][true_sentiments.index(true_s)],
                    "true_sentiment": true_s,
                    "predicted_sentiment": None,
                    "error_type": "Missing prediction"
                })

            elif true_s is None:
                FP += 1
                errors.append({
                    "id": row["id"],
                    "aspect": "EXTRA_PREDICTED",
                    "true_sentiment": None,
                    "predicted_sentiment": pred_s,
                    "error_type": "Extra prediction"
                })

            else:
                pred_norm = pred_s.strip().lower()

                if pred_norm not in valid_labels:
                    FN += 1
                    errors.append({
                        "id": row["id"],
                        "aspect": row["aspects"][true_sentiments.index(true_s)],
                        "true_sentiment": true_s,
                        "predicted_sentiment": pred_s,
                        "error_type": "Invalid label"
                    })

                elif pred_norm == true_s:
                    TP += 1

                else:
                    FP += 1
                    FN += 1
                    errors.append({
                        "id": row["id"],
                        "aspect": row["aspects"][true_sentiments.index(true_s)],
                        "true_sentiment": true_s,
                        "predicted_sentiment": pred_norm,
                        "error_type": "Incorrect label"
                    })

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall    = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1        = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    print(f"\nSalīdzināti {len(test_data)} teikumi (izlaisti {skipped})")
    print(f"True Positives (TP): {TP}")
    print(f"False Positives (FP): {FP}")
    print(f"False Negatives (FN): {FN}")
    print(f"\nPrecision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1 score:  {f1:.4f}")

    with open("errors_sentiment_labels.json", "w", encoding="utf-8") as f:
        json.dump(errors, f, ensure_ascii=False, indent=2)
    print("\nFP un FN kļūdu saraksts saglabāts uz errors_sentiment_labels.json")

test_data = parse_xml_sentiment_pred("LVtestGOLD_1.xml")
evaluate_sentiment_classification(test_data)


### VIENLAICĪGA ASPEKTU IZGŪŠANA UN NOSKAŅOJUMU NOTEIKŠANA

In [ ]:
# OpenRouter. Sample code and API for DeepSeek V3 0324 (free). Tiešsaiste. OpenRouter. Pieejams: https://openrouter.ai/deepseek/deepseek-chat-v3-0324:free/api. [skatīts 2025-03-29].
# OpenAI. ChatGPT o3-mini-high. Uzvedne: kā man uzlabot šo kodu, lai tas izmantotu no XML izgūtos teikumus, aspektus un noskaņojumus noskaņojuma noteikšanai aspektiem un vēl novērtētu modeļa veiktspēju? https://chatgpt.com/ [izmantots 2025-03-29]
# OpenAI. GPT-4o. Uzvedne: kā sasaistīt zemāk dotos kodus un izlabot manu kodu tā, lai pēc katras uzvednes tiktu izgūts rezultāts kurš uzreiz tiek salīdzināts ar attiecīgo pareizo atbildi, nesalīdzinot ar ID, un tad pareizi tiek aprēķinātas metrikas? https://chatgpt.com/ [izmantots 2025-04-24]

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import xml.etree.ElementTree as ET
import pandas as pd
import json
import re
import openai
from tqdm import tqdm

def parse_xml_full_absa(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    data = []
    for sentence in root.iter('sentence'):
        if "OutOfScope" in sentence.attrib and sentence.attrib["OutOfScope"].upper() == "TRUE":
          continue
        text = sentence.find('text').text.strip()
        opinions = sentence.find('Opinions')
        aspects = []
        sentiments = []
        if opinions is not None:
            for opinion in opinions.findall('Opinion'):
                target = opinion.get('target')
                polarity = opinion.get('polarity')
                if target and target.lower() != "null":
                  aspects.append(target.lower())
                  sentiments.append(polarity)
        data.append({
            'id': sentence.get('id'),
            'text': text,
            'aspects': aspects,
            'sentiments': sentiments
        })
    return pd.DataFrame(data)

client = openai.OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=""
)
def predict_sentiment(sentence_id, sentence):
    prompt = (
        "Your task is to do aspect based sentiment analysis.\n"
        "Given a sentence, extract all explicit aspect terms from a sentence and determine the sentiment towards each extracted aspect. Use only: positive, negative, neutral.\n"
        "Return the result as a JSON object with a single key called 'results', where the value is a list of [aspect, sentiment] pairs.\n"
        "Example: \n"
        "{\"results\": [[\"ēdiens\", \"positive\"], [\"apkalpošana\", \"neutral\"]]}\n"
        "If there are no aspect terms in the sentence, return an empty list for 'results'.\n"
        "{\"results\": []}\n\n"
        "Extract the aspect terms exactly as they appear in the sentence, preserving their original form and grammatical case.\n"
        "Do not provide any additional text or explanation and do not include implicit aspects or infer anything not explicitly stated.\n"
        "Do not include general concepts (e.g., 'cena', 'skaitlis', 'daudzums', 'pieredze').\n"
        "If the same aspect appears with mixed sentiment, repeat it for each sentiment (e.g., `baraviku zupa:positive, baraviku zupa:negative`)\n"
        "Do not use monetary values or pronouns (e.g., '11€', 'šīs') as aspects.\n"
        "Do not include any adjectives, verbs or general sentiment expressions.\n"
        "Respond ONLY with valid JSON. Do not add any text before or after.\n"
        f"Sentence: \"{sentence}\"\n"
    )
    try:
        completion = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model="gemma2-9b-it"
        )
        content = completion.choices[0].message.content.strip()
        match = re.search(r"\{.*\}", content, re.DOTALL)
        if not match:
            raise ValueError("No JSON in response.")
        parsed = json.loads(match.group(0))
        if isinstance(parsed, list):
          parsed = parsed[0]
        return parsed.get("results", [])

# OpenAI. GPT-4o. Uzvedne: varbūt būtu labāk šīs nepareizās atbildes ar exception kļūdām nesaglabāt vispār JSON failā bet kaut kur atsevišķi, jo tad vienkārši JSON failā nebūs kļūdainu rezultātu, kurus nevarēs nolasīt un ar šo rindiņu if sentence_id not in prediction_dict: uzreiz tiks uzskatīta kā kļūda https://chatgpt.com/ [izmantots 2025-04-24]
    except Exception as e:
        print(f"[ERROR] ID {sentence_id} — {e}")
        with open("llm_errors_full.log", "a", encoding="utf-8") as f:
            f.write(f"{sentence_id} — {e}\n")
            f.write(f"Original response: {completion}\n\n")
        return None

# OpenAI. GPT-4o. Uzvedne: kā izlabot manā kodā metriku aprēķināšanas funkciju vienlaicīgā aspektu izgūšanā un aspektu noskaņojuma noteikšanā, lai tiktu iegūti TP, FP un FN? aspekti var atkārtoties un tiem varbūt vienādi vai dažādi noskaņojumi https://chatgpt.com/ [izmantots 2025-04-26]
from collections import Counter

def evaluate_live(test_data):
    TP = 0
    FP = 0
    FN = 0
    skipped = 0
    fp_examples = []
    fn_examples = []

    for _, row in tqdm(test_data.iterrows(), total=len(test_data), desc="Processing"):
        true_aspects = [a.lower() for a in row["aspects"]]
        true_sentiments = [s.lower() for s in row["sentiments"]]

        pred_aspects = predict_sentiment(row["id"], row["text"])

        if pred_aspects is None:
            skipped += 1
            FN += len(true_aspects)
            continue

        pred_aspect_terms = [a[0].lower() for a in pred_aspects if isinstance(a, list) and len(a) == 2]
        pred_sentiments = [a[1].lower() for a in pred_aspects if isinstance(a, list) and len(a) == 2]

        true_counter = Counter(zip(true_aspects, true_sentiments))
        pred_counter = Counter(zip(pred_aspect_terms, pred_sentiments))

        for pair in (true_counter.keys() | pred_counter.keys()):
            true_count = true_counter.get(pair, 0)
            pred_count = pred_counter.get(pair, 0)

            TP += min(true_count, pred_count)
            if pred_count > true_count:
                FP += pred_count - true_count
                fp_examples.append({
                    "id": row["id"],
                    "text": row["text"],
                    "wrong_prediction": pair
                })
            if true_count > pred_count:
                FN += true_count - pred_count
                fn_examples.append({
                    "id": row["id"],
                    "text": row["text"],
                    "missing": pair
                })

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    print(f"\nSalīdzināti {len(test_data)} teikumi (izlaisti {skipped})")
    print(f"True Positives (TP): {TP}")
    print(f"False Positives (FP): {FP}")
    print(f"False Negatives (FN): {FN}")
    print(f"\nPrecision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1 score:  {f1:.4f}")

    with open("fp_examples.json", "w", encoding="utf-8") as f:
        json.dump(fp_examples, f, ensure_ascii=False, indent=2)

    with open("fn_examples.json", "w", encoding="utf-8") as f:
        json.dump(fn_examples, f, ensure_ascii=False, indent=2)

    print("\nFP un FN piemēri saglabāti 'fp_examples.json' un 'fn_examples.json'!")

test_data = parse_xml_full_absa("testGOLD2016_1.xml")
evaluate_live(test_data)

### VIENLAICĪGA ASPEKTU IZGŪŠANA UN NOSKAŅOJUMU NOTEIKŠANA AR PIEMĒRU

In [ ]:
# OpenRouter. Sample code and API for DeepSeek V3 0324 (free). Tiešsaiste. OpenRouter. Pieejams: https://openrouter.ai/deepseek/deepseek-chat-v3-0324:free/api. [skatīts 2025-03-29].
# OpenAI. ChatGPT o3-mini-high. Uzvedne: kā man uzlabot šo kodu, lai tas izmantotu no XML izgūtos teikumus, aspektus un noskaņojumus noskaņojuma noteikšanai aspektiem un vēl novērtētu modeļa veiktspēju? https://chatgpt.com/ [izmantots 2025-03-29]
# OpenAI. GPT-4o. Uzvedne: kā sasaistīt zemāk dotos kodus un izlabot manu kodu tā, lai pēc katras uzvednes tiktu izgūts rezultāts kurš uzreiz tiek salīdzināts ar attiecīgo pareizo atbildi, nesalīdzinot ar ID, un tad pareizi tiek aprēķinātas metrikas? https://chatgpt.com/ [izmantots 2025-04-24]

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import xml.etree.ElementTree as ET
import pandas as pd
import json
import re
import openai
from tqdm import tqdm

def parse_xml_full_absa(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    data = []
    for sentence in root.iter('sentence'):
        if "OutOfScope" in sentence.attrib and sentence.attrib["OutOfScope"].upper() == "TRUE":
          continue
        text = sentence.find('text').text.strip()
        opinions = sentence.find('Opinions')
        aspects = []
        sentiments = []
        if opinions is not None:
            for opinion in opinions.findall('Opinion'):
                target = opinion.get('target')
                polarity = opinion.get('polarity')
                if target and target.lower() != "null":
                  aspects.append(target.lower())
                  sentiments.append(polarity)
        data.append({
            'id': sentence.get('id'),
            'text': text,
            'aspects': aspects,
            'sentiments': sentiments
        })
    return pd.DataFrame(data)

client = openai.OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=""
)

def predict_sentiment(sentence_id, sentence):
    prompt = (
        "Your task is to do aspect based sentiment analysis.\n"
        "Given a sentence, extract all explicit aspect terms from a sentence and determine the sentiment towards each extracted aspect. Use only: positive, negative, neutral.\n"
        "Return the result as a JSON object with a single key called 'results', where the value is a list of [aspect, sentiment] pairs.\n"
        "Example: \n"
        "{\"results\": [[\"ēdiens\", \"positive\"], [\"apkalpošana\", \"neutral\"]]}\n"
        "If there are no aspect terms in the sentence, return an empty list for 'results'.\n"
        "{\"results\": []}\n\n"
        "Extract the aspect terms exactly as they appear in the sentence, preserving their original form and grammatical case.\n"
        "Do not provide any additional text or explanation and do not include implicit aspects or infer anything not explicitly stated.\n"
        "Do not include general concepts (e.g., 'cena', 'skaitlis', 'daudzums', 'pieredze').\n"
        "If the same aspect appears with mixed sentiment, repeat it for each sentiment (e.g., `baraviku zupa:positive, baraviku zupa:negative`)\n"
        "Do not use monetary values or pronouns (e.g., '11€', 'šīs') as aspects.\n"
        "Do not include any adjectives, verbs or general sentiment expressions.\n"
        "Respond ONLY with valid JSON. Do not add any text before or after.\n"
        "Example:\n"
        "Sentence: \"Ēdiens bija vidējs līdz virs vidējā; franču sīpolu zupa bija sātīga, bet nekas pārsteidzošs, un deserti nebija nevienā ziņā izcili.\"\n"
        "Answer: {\"results\": [[\"Ēdiens\", \"positive\"], [\"franču sīpolu zupa\", \"negative\"], [\"franču sīpolu zupa\", \"positive\"], [\"deserti\", \"negative\"]]}\n\n"
        f"Sentence: \"{sentence}\"\n"
    )
    try:
        completion = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model="gemma2-9b-it"
        )
        content = completion.choices[0].message.content.strip()
        match = re.search(r"\{.*\}", content, re.DOTALL)
        if not match:
            raise ValueError("No JSON in response.")
        parsed = json.loads(match.group(0))
        if isinstance(parsed, list):
          parsed = parsed[0]
        return parsed.get("results", [])

# OpenAI. GPT-4o. Uzvedne: varbūt būtu labāk šīs nepareizās atbildes ar exception kļūdām nesaglabāt vispār JSON failā bet kaut kur atsevišķi, jo tad vienkārši JSON failā nebūs kļūdainu rezultātu, kurus nevarēs nolasīt un ar šo rindiņu if sentence_id not in prediction_dict: uzreiz tiks uzskatīta kā kļūda https://chatgpt.com/ [izmantots 2025-04-24]
    except Exception as e:
        print(f"[ERROR] ID {sentence_id} — {e}")
        with open("llm_errors_full1.log", "a", encoding="utf-8") as f:
            f.write(f"{sentence_id} — {e}\n")
            f.write(f"Original response: {completion}\n\n")
        return None

# OpenAI. GPT-4o. Uzvedne: kā izlabot manā kodā metriku aprēķināšanas funkciju vienlaicīgā aspektu izgūšanā un aspektu noskaņojuma noteikšanā, lai tiktu iegūti TP, FP un FN? aspekti var atkārtoties un tiem varbūt vienādi vai dažādi noskaņojumi https://chatgpt.com/ [izmantots 2025-04-26]
from collections import Counter

def evaluate_live(test_data):
    TP = 0
    FP = 0
    FN = 0
    skipped = 0
    fp_examples = []
    fn_examples = []

    for _, row in tqdm(test_data.iterrows(), total=len(test_data), desc="Processing"):
        true_aspects = [a.lower() for a in row["aspects"]]
        true_sentiments = [s.lower() for s in row["sentiments"]]

        pred_aspects = predict_sentiment(row["id"], row["text"])

        if pred_aspects is None:
            skipped += 1
            FN += len(true_aspects)
            continue

        pred_aspect_terms = [a[0].lower() for a in pred_aspects if isinstance(a, list) and len(a) == 2]
        pred_sentiments = [a[1].lower() for a in pred_aspects if isinstance(a, list) and len(a) == 2]

        true_counter = Counter(zip(true_aspects, true_sentiments))
        pred_counter = Counter(zip(pred_aspect_terms, pred_sentiments))

        for pair in (true_counter.keys() | pred_counter.keys()):
            true_count = true_counter.get(pair, 0)
            pred_count = pred_counter.get(pair, 0)

            TP += min(true_count, pred_count)
            if pred_count > true_count:
                FP += pred_count - true_count
                fp_examples.append({
                    "id": row["id"],
                    "text": row["text"],
                    "wrong_prediction": pair
                })
            if true_count > pred_count:
                FN += true_count - pred_count
                fn_examples.append({
                    "id": row["id"],
                    "text": row["text"],
                    "missing": pair
                })

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    print(f"\nSalīdzināti {len(test_data)} teikumi (izlaisti {skipped})")
    print(f"True Positives (TP): {TP}")
    print(f"False Positives (FP): {FP}")
    print(f"False Negatives (FN): {FN}")
    print(f"\nPrecision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1 score:  {f1:.4f}")

    with open("fp_examples1.json", "w", encoding="utf-8") as f:
        json.dump(fp_examples, f, ensure_ascii=False, indent=2)

    with open("fn_examples1.json", "w", encoding="utf-8") as f:
        json.dump(fn_examples, f, ensure_ascii=False, indent=2)

    print("\nFP un FN piemēri saglabāti 'fp_examples.json' un 'fn_examples.json'!")

test_data = parse_xml_full_absa("LVtestGOLD_1.xml")
evaluate_live(test_data)

### ASPEKTU IZGŪŠANA AR LEMMATIZĀCIJU

In [ ]:
# OpenAI. GPT-4o. Uzvedne: kā sasaistīt zemāk dotos kodus un izlabot manu kodu tā, lai pēc katras uzvednes tiktu izgūts rezultāts kurš uzreiz tiek salīdzināts ar attiecīgo pareizo atbildi, nesalīdzinot ar ID, un tad pareizi tiek aprēķinātas metrikas? https://chatgpt.com/ [izmantots 2025-04-24]
# OpenAI. GPT-4o. Uzvedne: lemmatizāciju tā, lai lemmatizācija aspektiem tiktu veikta pašā pārbaudē, kur tiek skaitīti TP FP FN? https://chatgpt.com/ [izmantots 2025-05-08]

import stanza
stanza.download("lv")
nlp = stanza.Pipeline("lv", processors="tokenize,pos,lemma", use_gpu=False)
def lemmatize_phrase(phrase):
    doc = nlp(phrase)
    return " ".join([word.lemma.lower() for sent in doc.sentences for word in sent.words])


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import xml.etree.ElementTree as ET
import pandas as pd
import json
import re
import openai
from tqdm import tqdm

# OpenRouter. Sample code and API for DeepSeek V3 0324 (free). Tiešsaiste. OpenRouter. Pieejams: https://openrouter.ai/deepseek/deepseek-chat-v3-0324:free/api. [skatīts 2025-03-29].
# OpenAI. ChatGPT o3-mini-high. Uzvedne: kā man uzlabot šo kodu, lai tas izmantotu no XML izgūtos teikumus, aspektus un noskaņojumus noskaņojuma noteikšanai aspektiem un vēl novērtētu modeļa veiktspēju? https://chatgpt.com/ [izmantots 2025-03-29]

def parse_xml_aspect_extraction(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    data = []
    for sentence in root.iter('sentence'):
        if "OutOfScope" in sentence.attrib and sentence.attrib["OutOfScope"].upper() == "TRUE":
          continue
        text = sentence.find('text').text.strip()
        opinions = sentence.find('Opinions')
        aspects = []
        if opinions is not None:
            for opinion in opinions.findall('Opinion'):
                target = opinion.get('target')
                if target and target.lower() != "null":
                  aspects.append(target.lower())
        data.append({
            'id': sentence.get('id'),
            'text': text,
            'aspects': aspects
        })
    return pd.DataFrame(data)

client = openai.OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=""
)
def extract_aspects(sentence_id, sentence):
    prompt = (
        "Your task is to do aspect term extraction.\n"
        "Given a sentence, extract all explicit aspect terms from a sentence.\n"
        "Return the result as a single JSON object with key 'results'.\n"
        "If there are no aspect terms in the sentence, return an empty list for 'results'.\n"
        "Extract the aspect terms exactly as they appear in the sentence, preserving their original form and grammatical case.\n"
        "Do not provide any additional text or explanation, do not guess implicit aspects, do not extract adjectives, verbs or general sentiment expressions.\n"
        "Do not include broad or vague terms (e.g., 'cena', 'diena') and always preserve the full noun phrase (e.g., 'ķiploku grauzdiņi', not just 'grauzdiņi')\n"
        "Respond ONLY with valid JSON.\n"
        "Example format:\n"
        "{\"results\": [\"ēdiens\", \"apkalpošana\"]}\n"
        "{\"results\": []}\n\n"
        f"Sentence: \"{sentence}\""
    )
    try:
        completion = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model="qwen-qwq-32b"
        )
        content = completion.choices[0].message.content.strip()
        match = re.search(r"\{.*\}", content, re.DOTALL)
        if not match:
            raise ValueError("No JSON in response.")
        parsed = json.loads(match.group(0))
        return parsed.get("results", [])

# OpenAI. GPT-4o. Uzvedne: varbūt būtu labāk šīs nepareizās atbildes ar exception kļūdām nesaglabāt vispār JSON failā bet kaut kur atsevišķi, jo tad vienkārši JSON failā nebūs kļūdainu rezultātu, kurus nevarēs nolasīt un ar šo rindiņu if sentence_id not in prediction_dict: uzreiz tiks uzskatīta kā kļūda https://chatgpt.com/ [izmantots 2025-04-24]
    except Exception as e:
        print(f"[ERROR] ID {sentence_id} — {e}")
        with open("llm_errors_aspects.log", "a", encoding="utf-8") as f:
            f.write(f"{sentence_id} — {e}\n")
            f.write(f"Original response: {completion}\n\n")
        return None

# OpenAI. GPT-4o. Uzvedne: kā izlabot manā kodā metriku aprēķināšanas funkciju, lai tiktu iegūti TP, FP un FN? https://chatgpt.com/ [izmantots 2025-04-26]

def evaluate_live(test_data):
    TP = 0
    FP = 0
    FN = 0
    skipped = 0
    mismatches = []

    for _, row in tqdm(test_data.iterrows(), total=len(test_data), desc="Processing"):

        current_TP = 0
        current_FP = 0
        current_FN = 0

        sentence_id = row["id"]
        sentence = row["text"]
        true_aspects = [lemmatize_phrase(a.lower()) for a in row["aspects"]]
        pred_aspects = extract_aspects(row["id"], row["text"])


        if pred_aspects is None:
            skipped += 1
            FN += len(true_aspects)
            continue

        pred_aspects = [lemmatize_phrase(a.lower()) for a in pred_aspects]

        true_set = set(true_aspects)
        pred_set = set(pred_aspects)

        current_TP += len(true_set & pred_set)   # Pareizi atrastie aspekti
        current_FP += len(pred_set - true_set)   # Nepareizi atrastie aspekti (kļūdaini izdomāti)
        current_FN += len(true_set - pred_set)   # Pazudušie aspekti (neatrastie)

        TP += current_TP
        FP += current_FP
        FN += current_FN

        if current_FP > 0 or current_FN > 0:
            mismatches.append({
                'id': sentence_id,
                'text': sentence,
                'true_aspects': true_aspects,
                'predicted_aspects': pred_aspects,
                'FP': list(pred_set - true_set),
                'FN': list(true_set - pred_set)
            })

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    print(f"\nSalīdzināti {len(test_data)} teikumi (izlaisti {skipped})")
    print(f"True Positives (TP): {TP}")
    print(f"False Positives (FP): {FP}")
    print(f"False Negatives (FN): {FN}")
    print(f"\nPrecision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1 score:  {f1:.4f}")

    with open("errors_fp_fn.json", "w", encoding="utf-8") as f:
        json.dump(mismatches, f, ensure_ascii=False, indent=2)

    print("\nFP un FN kļūdu saraksts saglabāts uz 'errors_fp_fn.json'")

test_data = parse_xml_aspect_extraction("testGOLD2016_1.xml")
evaluate_live(test_data)

### KOMBINĒTAIS UZDEVUMS AR LEMMATIZĀCIJU

In [ ]:
# OpenRouter. Sample code and API for DeepSeek V3 0324 (free). Tiešsaiste. OpenRouter. Pieejams: https://openrouter.ai/deepseek/deepseek-chat-v3-0324:free/api. [skatīts 2025-03-29].
# OpenAI. ChatGPT o3-mini-high. Uzvedne: kā man uzlabot šo kodu, lai tas izmantotu no XML izgūtos teikumus, aspektus un noskaņojumus noskaņojuma noteikšanai aspektiem un vēl novērtētu modeļa veiktspēju? https://chatgpt.com/ [izmantots 2025-03-29]
# OpenAI. GPT-4o. Uzvedne: kā sasaistīt zemāk dotos kodus un izlabot manu kodu tā, lai pēc katras uzvednes tiktu izgūts rezultāts kurš uzreiz tiek salīdzināts ar attiecīgo pareizo atbildi, nesalīdzinot ar ID, un tad pareizi tiek aprēķinātas metrikas? https://chatgpt.com/ [izmantots 2025-04-24]
# OpenAI. GPT-4o. Uzvedne: lemmatizāciju tā, lai lemmatizācija aspektiem tiktu veikta pašā pārbaudē, kur tiek skaitīti TP FP FN? https://chatgpt.com/ [izmantots 2025-05-08]

import stanza
stanza.download("lv")
nlp = stanza.Pipeline("lv", processors="tokenize,pos,lemma", use_gpu=False)
def lemmatize_phrase(phrase):
    doc = nlp(phrase)
    return " ".join([word.lemma.lower() for sent in doc.sentences for word in sent.words])

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import xml.etree.ElementTree as ET
import pandas as pd
import json
import re
import openai
from tqdm import tqdm

def parse_xml_full_absa(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    data = []
    for sentence in root.iter('sentence'):
        if "OutOfScope" in sentence.attrib and sentence.attrib["OutOfScope"].upper() == "TRUE":
          continue
        text = sentence.find('text').text.strip()
        opinions = sentence.find('Opinions')
        aspects = []
        sentiments = []
        if opinions is not None:
            for opinion in opinions.findall('Opinion'):
                target = opinion.get('target')
                polarity = opinion.get('polarity')
                if target and target.lower() != "null":
                  aspects.append(target.lower())
                  sentiments.append(polarity)
        data.append({
            'id': sentence.get('id'),
            'text': text,
            'aspects': aspects,
            'sentiments': sentiments
        })
    return pd.DataFrame(data)

client = openai.OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=""
)

def predict_sentiment(sentence_id, sentence):
    prompt = (
        "Your task is to do aspect based sentiment analysis.\n"
        "Given a sentence, extract all explicit aspect terms from a sentence and determine the sentiment towards each extracted aspect. Use only: positive, negative, neutral.\n"
        "Return the result as a JSON object with a single key called 'results', where the value is a list of [aspect, sentiment] pairs.\n"
        "Example: \n"
        "{\"results\": [[\"ēdiens\", \"positive\"], [\"apkalpošana\", \"neutral\"]]}\n"
        "If there are no aspect terms in the sentence, return an empty list for 'results'.\n"
        "{\"results\": []}\n\n"
        "Extract the aspect terms exactly as they appear in the sentence, preserving their original form and grammatical case.\n"
        "Do not provide any additional text or explanation and do not include implicit aspects or infer anything not explicitly stated.\n"
        "Do not include general concepts (e.g., 'cena', 'skaitlis', 'daudzums', 'pieredze').\n"
        "If the same aspect appears with mixed sentiment, repeat it for each sentiment (e.g., `baraviku zupa:positive, baraviku zupa:negative`)\n"
        "Do not use monetary values or pronouns (e.g., '11€', 'šīs') as aspects.\n"
        "Do not include any adjectives, verbs or general sentiment expressions.\n"
        "Respond ONLY with valid JSON. Do not add any text before or after.\n"
        f"Sentence: \"{sentence}\"\n"
    )
    try:
        completion = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model="gemma2-9b-it"
        )
        content = completion.choices[0].message.content.strip()
        match = re.search(r"\{.*\}", content, re.DOTALL)
        if not match:
            raise ValueError("No JSON in response.")
        parsed = json.loads(match.group(0))
        if isinstance(parsed, list):
          parsed = parsed[0]
        return parsed.get("results", [])

# OpenAI. GPT-4o. Uzvedne: varbūt būtu labāk šīs nepareizās atbildes ar exception kļūdām nesaglabāt vispār JSON failā bet kaut kur atsevišķi, jo tad vienkārši JSON failā nebūs kļūdainu rezultātu, kurus nevarēs nolasīt un ar šo rindiņu if sentence_id not in prediction_dict: uzreiz tiks uzskatīta kā kļūda https://chatgpt.com/ [izmantots 2025-04-24]
    except Exception as e:
        print(f"[ERROR] ID {sentence_id} — {e}")
        with open("llm_errors_full.log", "a", encoding="utf-8") as f:
            f.write(f"{sentence_id} — {e}\n")
            f.write(f"Original response: {completion}\n\n")
        return None

# OpenAI. GPT-4o. Uzvedne: kā izlabot manā kodā metriku aprēķināšanas funkciju vienlaicīgā aspektu izgūšanā un aspektu noskaņojuma noteikšanā, lai tiktu iegūti TP, FP un FN? aspekti var atkārtoties un tiem varbūt vienādi vai dažādi noskaņojumi https://chatgpt.com/ [izmantots 2025-04-26]
from collections import Counter

def evaluate_live(test_data):
    TP = 0
    FP = 0
    FN = 0
    skipped = 0
    fp_examples = []
    fn_examples = []

    for _, row in tqdm(test_data.iterrows(), total=len(test_data), desc="Processing"):
        true_pairs = [
            (lemmatize_phrase(a.lower()), s.lower())
            for a, s in zip(row["aspects"], row["sentiments"])
        ]

        pred_aspects = predict_sentiment(row["id"], row["text"])

        if pred_aspects is None:
            skipped += 1
            FN += len(true_pairs)
            continue

        pred_pairs = [
            (lemmatize_phrase(a[0].lower()), a[1].lower())
            for a in pred_aspects if isinstance(a, list) and len(a) == 2
        ]

        true_counter = Counter(true_pairs)
        pred_counter = Counter(pred_pairs)

        for pair in (true_counter.keys() | pred_counter.keys()):
            true_count = true_counter.get(pair, 0)
            pred_count = pred_counter.get(pair, 0)

            TP += min(true_count, pred_count)
            if pred_count > true_count:
                FP += pred_count - true_count
                fp_examples.append({
                    "id": row["id"],
                    "text": row["text"],
                    "wrong_prediction": pair
                })
            if true_count > pred_count:
                FN += true_count - pred_count
                fn_examples.append({
                    "id": row["id"],
                    "text": row["text"],
                    "missing": pair
                })

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    print(f"\nSalīdzināti {len(test_data)} teikumi (izlaisti {skipped})")
    print(f"True Positives (TP): {TP}")
    print(f"False Positives (FP): {FP}")
    print(f"False Negatives (FN): {FN}")
    print(f"\nPrecision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1 score:  {f1:.4f}")

    with open("fp_examples.json", "w", encoding="utf-8") as f:
        json.dump(fp_examples, f, ensure_ascii=False, indent=2)

    with open("fn_examples.json", "w", encoding="utf-8") as f:
        json.dump(fn_examples, f, ensure_ascii=False, indent=2)

    print("\nFP un FN piemēri saglabāti 'fp_examples.json' un 'fn_examples.json'!")

test_data = parse_xml_full_absa("LVtestGOLD_1.xml")
evaluate_live(test_data)

### KOMBINĒTAIS UZDEVUMS AR PIEMĒRU UN AR LEMMATIZĀCIJU

In [ ]:
# OpenRouter. Sample code and API for DeepSeek V3 0324 (free). Tiešsaiste. OpenRouter. Pieejams: https://openrouter.ai/deepseek/deepseek-chat-v3-0324:free/api. [skatīts 2025-03-29].
# OpenAI. ChatGPT o3-mini-high. Uzvedne: kā man uzlabot šo kodu, lai tas izmantotu no XML izgūtos teikumus, aspektus un noskaņojumus noskaņojuma noteikšanai aspektiem un vēl novērtētu modeļa veiktspēju? https://chatgpt.com/ [izmantots 2025-03-29]
# OpenAI. GPT-4o. Uzvedne: kā sasaistīt zemāk dotos kodus un izlabot manu kodu tā, lai pēc katras uzvednes tiktu izgūts rezultāts kurš uzreiz tiek salīdzināts ar attiecīgo pareizo atbildi, nesalīdzinot ar ID, un tad pareizi tiek aprēķinātas metrikas? https://chatgpt.com/ [izmantots 2025-04-24]
# OpenAI. GPT-4o. Uzvedne: lemmatizāciju tā, lai lemmatizācija aspektiem tiktu veikta pašā pārbaudē, kur tiek skaitīti TP FP FN? https://chatgpt.com/ [izmantots 2025-05-08]

import stanza
stanza.download("lv")
nlp = stanza.Pipeline("lv", processors="tokenize,pos,lemma", use_gpu=False)
def lemmatize_phrase(phrase):
    doc = nlp(phrase)
    return " ".join([word.lemma.lower() for sent in doc.sentences for word in sent.words])

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import xml.etree.ElementTree as ET
import pandas as pd
import json
import re
import openai
from tqdm import tqdm

def parse_xml_full_absa(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    data = []
    for sentence in root.iter('sentence'):
        if "OutOfScope" in sentence.attrib and sentence.attrib["OutOfScope"].upper() == "TRUE":
          continue
        text = sentence.find('text').text.strip()
        opinions = sentence.find('Opinions')
        aspects = []
        sentiments = []
        if opinions is not None:
            for opinion in opinions.findall('Opinion'):
                target = opinion.get('target')
                polarity = opinion.get('polarity')
                if target and target.lower() != "null":
                  aspects.append(target.lower())
                  sentiments.append(polarity)
        data.append({
            'id': sentence.get('id'),
            'text': text,
            'aspects': aspects,
            'sentiments': sentiments
        })
    return pd.DataFrame(data)

client = openai.OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=""
)

def predict_sentiment(sentence_id, sentence):
    prompt = (
        "Your task is to do aspect based sentiment analysis.\n"
        "Given a sentence, extract all explicit aspect terms from a sentence and determine the sentiment towards each extracted aspect. Use only: positive, negative, neutral.\n"
        "Return the result as a JSON object with a single key called 'results', where the value is a list of [aspect, sentiment] pairs.\n"
        "Example: \n"
        "{\"results\": [[\"ēdiens\", \"positive\"], [\"apkalpošana\", \"neutral\"]]}\n"
        "If there are no aspect terms in the sentence, return an empty list for 'results'.\n"
        "{\"results\": []}\n\n"
        "Extract the aspect terms exactly as they appear in the sentence, preserving their original form and grammatical case.\n"
        "Do not provide any additional text or explanation and do not include implicit aspects or infer anything not explicitly stated.\n"
        "Do not include general concepts (e.g., 'cena', 'skaitlis', 'daudzums', 'pieredze').\n"
        "If the same aspect appears with mixed sentiment, repeat it for each sentiment (e.g., `baraviku zupa:positive, baraviku zupa:negative`)\n"
        "Do not use monetary values or pronouns (e.g., '11€', 'šīs') as aspects.\n"
        "Do not include any adjectives, verbs or general sentiment expressions.\n"
        "Respond ONLY with valid JSON. Do not add any text before or after.\n"
        "Example:\n"
        "Sentence: \"Ēdiens bija vidējs līdz virs vidējā; franču sīpolu zupa bija sātīga, bet nekas pārsteidzošs, un deserti nebija nevienā ziņā izcili.\"\n"
        "Answer: {\"results\": [[\"Ēdiens\", \"positive\"], [\"franču sīpolu zupa\", \"negative\"], [\"franču sīpolu zupa\", \"positive\"], [\"deserti\", \"negative\"]]}\n\n"
        f"Sentence: \"{sentence}\"\n"
    )
    try:
        completion = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model="gemma2-9b-it"
        )
        content = completion.choices[0].message.content.strip()
        match = re.search(r"\{.*\}", content, re.DOTALL)
        if not match:
            raise ValueError("No JSON in response.")
        parsed = json.loads(match.group(0))
        if isinstance(parsed, list):
          parsed = parsed[0]
        return parsed.get("results", [])

# OpenAI. GPT-4o. Uzvedne: varbūt būtu labāk šīs nepareizās atbildes ar exception kļūdām nesaglabāt vispār JSON failā bet kaut kur atsevišķi, jo tad vienkārši JSON failā nebūs kļūdainu rezultātu, kurus nevarēs nolasīt un ar šo rindiņu if sentence_id not in prediction_dict: uzreiz tiks uzskatīta kā kļūda https://chatgpt.com/ [izmantots 2025-04-24]
    except Exception as e:
        print(f"[ERROR] ID {sentence_id} — {e}")
        with open("llm_errors_full.log", "a", encoding="utf-8") as f:
            f.write(f"{sentence_id} — {e}\n")
            f.write(f"Original response: {completion}\n\n")
        return None

# OpenAI. GPT-4o. Uzvedne: kā izlabot manā kodā metriku aprēķināšanas funkciju vienlaicīgā aspektu izgūšanā un aspektu noskaņojuma noteikšanā, lai tiktu iegūti TP, FP un FN? aspekti var atkārtoties un tiem varbūt vienādi vai dažādi noskaņojumi https://chatgpt.com/ [izmantots 2025-04-26]
from collections import Counter

def evaluate_live(test_data):
    TP = 0
    FP = 0
    FN = 0
    skipped = 0
    fp_examples = []
    fn_examples = []

    for _, row in tqdm(test_data.iterrows(), total=len(test_data), desc="Processing"):
        true_pairs = [
            (lemmatize_phrase(a.lower()), s.lower())
            for a, s in zip(row["aspects"], row["sentiments"])
        ]

        pred_aspects = predict_sentiment(row["id"], row["text"])

        if pred_aspects is None:
            skipped += 1
            FN += len(true_pairs)
            continue

        pred_pairs = [
            (lemmatize_phrase(a[0].lower()), a[1].lower())
            for a in pred_aspects if isinstance(a, list) and len(a) == 2
        ]

        true_counter = Counter(true_pairs)
        pred_counter = Counter(pred_pairs)

        for pair in (true_counter.keys() | pred_counter.keys()):
            true_count = true_counter.get(pair, 0)
            pred_count = pred_counter.get(pair, 0)

            TP += min(true_count, pred_count)
            if pred_count > true_count:
                FP += pred_count - true_count
                fp_examples.append({
                    "id": row["id"],
                    "text": row["text"],
                    "wrong_prediction": pair
                })
            if true_count > pred_count:
                FN += true_count - pred_count
                fn_examples.append({
                    "id": row["id"],
                    "text": row["text"],
                    "missing": pair
                })

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    print(f"\nSalīdzināti {len(test_data)} teikumi (izlaisti {skipped})")
    print(f"True Positives (TP): {TP}")
    print(f"False Positives (FP): {FP}")
    print(f"False Negatives (FN): {FN}")
    print(f"\nPrecision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1 score:  {f1:.4f}")

    with open("fp_examples.json", "w", encoding="utf-8") as f:
        json.dump(fp_examples, f, ensure_ascii=False, indent=2)

    with open("fn_examples.json", "w", encoding="utf-8") as f:
        json.dump(fn_examples, f, ensure_ascii=False, indent=2)

    print("\nFP un FN piemēri saglabāti 'fp_examples.json' un 'fn_examples.json'!")

test_data = parse_xml_full_absa("LVtestGOLD_1.xml")
evaluate_live(test_data)